<a href="https://colab.research.google.com/github/BezymiannyiOlexii/test_smartys/blob/main/test_smartys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
from scipy.spatial.distance import cosine
import numpy as np
from sklearn.svm import SVC

# det data from GDisk

In [ ]:
!pip install PyDrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
downloaded = drive.CreateFile({'id':"19fiBFAsRKzoxuTn_dRY1zRPbdl6qbecr"})
downloaded.GetContentFile('test.tsv')
downloaded = drive.CreateFile({'id':"1b7AY_ULK3e59ywEJmHn-gX9E0T5Wn4EF"})
downloaded.GetContentFile('train.tsv')  

In [6]:
!rm /content/adc.json

# parsing data

In [7]:
df_test = pd.read_csv('test.tsv', header=0, sep='\t')
df_train = pd.read_csv('train.tsv', header=0, sep='\t')

In [8]:
print(f'size of train ds is : {df_train.shape[0]}, size of test ds is : {df_test.shape[0]}')

size of train ds is : 15585, size of test ds is : 1949


In [9]:
df_train.head()

,embedding_x,embedding_y,equal
0,"0.35198048,0.23884374,0.20589112,0.033522803,0...","0.124149986,0.14133812,0.16054663,0.38359353,0...",1
1,"0.3492841,-0.18005244,0.46332806,-0.2122336,0....","-0.7460901,-0.21427503,-0.14653064,-0.22343078...",0
2,"0.5642844,-0.08399645,-0.018092873,0.18461637,...","-0.30552676,0.10498648,0.40437204,0.25560898,-...",0
3,"0.36132073,-0.20658037,0.4909494,0.12876663,0....","0.25936005,-0.15755327,0.4929783,-0.11315008,0...",1
4,"-0.039740197,-0.11732757,-0.07907649,0.1649697...","0.26061502,-0.31516105,-0.04165541,0.16747917,...",1


In [10]:
#parsing data by converting str to list of float numbers
for df in (df_test, df_train):
  for column_name in df.columns.values.tolist()[:2]:
    for index in range(df.shape[0]):
      df[column_name][index] = [float(numb) for numb in df[column_name][index].split(',')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


# cosine

The data is normalized, therefore dot product and cosine_similarity is equal. Pearson correlation is cosine similarity between centered vectors (standart normilezed) thats why differnce between this metricts won't be statistically significant)

In [11]:
def add_cosine_column(df):
  cosine_lst = []
  for index in range(df.shape[0]):
    similarity = 1. - cosine(df.loc[index, 'embedding_x'],
                                  df.loc[index, 'embedding_y'])
    cosine_lst.append(similarity)
  df['cosine'] = cosine_lst

In [12]:
add_cosine_column(df_train)

In [13]:
df_train.head(5)

,embedding_x,embedding_y,equal,cosine
0,"[0.35198048, 0.23884374, 0.20589112, 0.0335228...","[0.124149986, 0.14133812, 0.16054663, 0.383593...",1,0.557235
1,"[0.3492841, -0.18005244, 0.46332806, -0.212233...","[-0.7460901, -0.21427503, -0.14653064, -0.2234...",0,-0.327108
2,"[0.5642844, -0.08399645, -0.018092873, 0.18461...","[-0.30552676, 0.10498648, 0.40437204, 0.255608...",0,-0.278190
3,"[0.36132073, -0.20658037, 0.4909494, 0.1287666...","[0.25936005, -0.15755327, 0.4929783, -0.113150...",1,0.772270
4,"[-0.039740197, -0.11732757, -0.07907649, 0.164...","[0.26061502, -0.31516105, -0.04165541, 0.16747...",1,0.445477


# classfication problem

In [14]:
X_train = df_train.loc[:, 'cosine'].to_numpy()
y_train = df_train.loc[:, 'equal'].to_numpy()

In [15]:
svc_model = SVC(kernel='linear', random_state=32)
svc_model.fit(X_train.reshape(-1,1), y_train)

SVC(kernel='linear', random_state=32)

In [17]:
threshold = -svc_model.intercept_[0]/svc_model.coef_[0][0]

In [20]:
print(threshold)

0.27216407871513515


#validation

In [18]:
add_cosine_column(df_test)

In [19]:
df_test.head()

,embedding_x,embedding_y,equal,cosine
0,"[0.297163, -0.19147852, 0.514213, 0.11966697, ...","[0.33785084, -0.16910027, -0.1574438, -0.02361...",0,0.034470
1,"[0.28278404, -0.21183152, 0.3420205, -0.230233...","[0.297163, -0.19147852, 0.514213, 0.11966697, ...",1,0.692232
2,"[0.50050944, -0.07608504, 0.1397811, 0.1756001...","[0.40801406, 0.10747076, 0.3100398, 0.04043074...",1,0.489577
3,"[0.18524958, -0.23291345, 0.47159392, 0.213189...","[-0.04595025, -0.4184819, -0.3319743, -0.18005...",1,0.102176
4,"[0.3160543, -0.12296223, 0.27941337, 0.2727139...","[0.40801406, 0.10747076, 0.3100398, 0.04043074...",1,0.448001


In [20]:
X_test = df_test.loc[:, 'cosine'].to_numpy()
y_test = df_test.loc[:, 'equal'].to_numpy()

In [21]:
y_pred = list(map(lambda x: 1 if x > threshold else 0, X_test))

In [22]:
def validation(y_pred, y_test):
  _is_equal_lists = [1 for i, j in zip(y_pred, y_test) if i == j]
  accuracy = sum(_is_equal_lists)/len(y_pred)
  return accuracy

In [23]:
accuracy = validation(y_pred, y_test)

In [24]:
print(accuracy)

0.8927655207798871
